In [1]:
import os

entries = os.listdir('./pictures')

In [2]:
ls

 Volume in drive C has no label.
 Volume Serial Number is 2475-1BCE

 Directory of C:\Users\varun\Desktop\WhatsApp

09-Mar-20  11:04 AM    <DIR>          .
09-Mar-20  11:04 AM    <DIR>          ..
08-Mar-20  08:51 PM    <DIR>          .ipynb_checkpoints
08-Mar-20  08:06 PM         8,144,835 My_rotation_matrix.bin
09-Mar-20  11:08 AM    <DIR>          pictures
08-Mar-20  01:13 PM        34,408,672 seperate.html
08-Mar-20  01:06 PM        39,336,194 seperate.ipynb
08-Mar-20  01:26 PM             7,626 seperate.py
08-Mar-20  09:21 PM            28,036 seperate1.ipynb
               5 File(s)     81,925,363 bytes
               4 Dir(s)  99,390,148,608 bytes free


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Load an color image in grayscale
for images in entries:
    img=mpimg.imread(path+'/'+images)
#     print(img.shape)
    

In [0]:
import torchvision.models as models
model = models.resnet18(pretrained=True)
layer = model._modules.get('avgpool')
model.eval()

In [0]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.imagenet_utils import decode_predictions
import matplotlib.pyplot as plt
%matplotlib inline

filename = path+'/IMG-20160224-WA0006.jpg'
# load an image in PIL format
original = load_img(filename, target_size=(224, 224))
print('PIL image size',original.size)
plt.imshow(original)
plt.show()

# convert the PIL image to a numpy array
# IN PIL - image is in (width, height, channel)
# In Numpy - image is in (height, width, channel)
numpy_image = img_to_array(original)
plt.imshow(np.uint8(numpy_image))
plt.show()
print('numpy array size',numpy_image.shape)

# Convert the image / images into batch format
# expand_dims will add an extra dimension to the data at a particular axis
# We want the input matrix to the network to be of the form (batchsize, height, width, channels)
# Thus we add the extra dimension to the axis 0.
image_batch = np.expand_dims(numpy_image, axis=0)
print('image batch size', image_batch.shape)
plt.imshow(np.uint8(image_batch[0]))

In [0]:
# prepare the image for the VGG model
processed_image = vgg16.preprocess_input(image_batch.copy())

# get the predicted probabilities for each class
predictions = vgg_model.predict(processed_image)
# print predictions

# convert the probabilities to class labels
# We will get top 5 predictions which is the default
label = decode_predictions(predictions)
print(label)

In [3]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import numpy as np

class Img2Vec():

    def __init__(self, cuda=False, model='resnet-18', layer='default', layer_output_size=512):
        """ Img2Vec
        :param cuda: If set to True, will run forward pass on GPU
        :param model: String name of requested model
        :param layer: String or Int depending on model.  See more docs: https://github.com/christiansafka/img2vec.git
        :param layer_output_size: Int depicting the output size of the requested layer
        """
        self.device = torch.device("cuda" if cuda else "cpu")
        self.layer_output_size = layer_output_size
        self.model_name = model
        
        self.model, self.extraction_layer = self._get_model_and_layer(model, layer)

        self.model = self.model.to(self.device)

        self.model.eval()

        self.scaler = transforms.Scale((224, 224))
        self.normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                              std=[0.229, 0.224, 0.225])
        self.to_tensor = transforms.ToTensor()

    def get_vec(self, img, tensor=False):
        """ Get vector embedding from PIL image
        :param img: PIL Image or list of PIL Images
        :param tensor: If True, get_vec will return a FloatTensor instead of Numpy array
        :returns: Numpy ndarray
        """
        if type(img) == list:
            a = [self.normalize(self.to_tensor(self.scaler(im))) for im in img]
            images = torch.stack(a).to(self.device) 
            if self.model_name == 'alexnet':
                my_embedding = torch.zeros(len(img), self.layer_output_size)
            else:
                my_embedding = torch.zeros(len(img), self.layer_output_size, 1, 1)

            def copy_data(m, i, o):
                my_embedding.copy_(o.data)

            h = self.extraction_layer.register_forward_hook(copy_data)
            h_x = self.model(images)
            h.remove()

            if tensor:
                return my_embedding
            else:
                if self.model_name == 'alexnet':
                    return my_embedding.numpy()[:, :]
                else:
                    print(my_embedding.numpy()[:, :, 0, 0].shape)
                    return my_embedding.numpy()[:, :, 0, 0]
        else:
            image = self.normalize(self.to_tensor(self.scaler(img))).unsqueeze(0).to(self.device)

            if self.model_name == 'alexnet':
                my_embedding = torch.zeros(1, self.layer_output_size)
            else:
                my_embedding = torch.zeros(1, self.layer_output_size, 1, 1)

            def copy_data(m, i, o):
                my_embedding.copy_(o.data)

            h = self.extraction_layer.register_forward_hook(copy_data)
            h_x = self.model(image)
            h.remove()

            if tensor:
                return my_embedding
            else:
                if self.model_name == 'alexnet':
                    return my_embedding.numpy()[0, :]
                else:
                    return my_embedding.numpy()[0, :, 0, 0]

    def _get_model_and_layer(self, model_name, layer):
        """ Internal method for getting layer from model
        :param model_name: model name such as 'resnet-18'
        :param layer: layer as a string for resnet-18 or int for alexnet
        :returns: pytorch model, selected layer
        """
        if model_name == 'resnet-18':
            model = models.resnet18(pretrained=True)
            if layer == 'default':
                layer = model._modules.get('avgpool')
                self.layer_output_size = 512
            else:
                layer = model._modules.get(layer)

            return model, layer

        elif model_name == 'alexnet':
            model = models.alexnet(pretrained=True)
            if layer == 'default':
                layer = model.classifier[-2]
                self.layer_output_size = 4096
            else:
                layer = model.classifier[-layer]

            return model, layer

        else:
            raise KeyError('Model %s was not found' % model_name)



In [15]:
import pdb
import os

input_path=path
entries = os.listdir('./Media/WhatsApp Images')
# entries

NameError: ignored

In [4]:
img2vec = Img2Vec()
from PIL import Image
# For each test image, we store the filename and vector as key, value in a dictionary
pics = {}
i=0
for file in os.listdir('./pictures'):
    filename = os.fsdecode(file)
    img = Image.open(os.path.join('./pictures', filename))
    vec = img2vec.get_vec(img)
    pics[filename] = vec    
    i=i+1
    if(i%200==0):
        print(i)



C:\Users\varun\Miniconda3\envs\test_env\lib\site-packages\torchvision\transforms\transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [0]:
import pickle
with open("My_rotation_matrix.bin", "wb") as output:
    pickle.dump(pics, output)

In [4]:
import pickle
with open("My_rotation_matrix.bin", "rb") as data:
    pics = pickle.load(data)

In [5]:
vector_key=[]
for key in pics:
  # print(key,pics[key])
  vector_key+=[pics[key]]
  # break

In [6]:
len(vector_key)

160

In [7]:
n_clusters=2 
from sklearn.cluster import KMeans
Kmean = KMeans(n_clusters=5)
Kmean.fit(vector_key)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [8]:
Kmean.labels_

array([0, 4, 4, 0, 4, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 0,
       0, 4, 4, 4, 4, 0, 0, 0, 4, 3, 3, 4, 4, 2, 2, 2, 2, 2, 2, 2, 4, 2,
       2, 2, 2, 2, 4, 4, 4, 4, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4,
       4, 4, 3, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 4, 4, 3, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [10]:
n_clusters=5 

# Python program to explain os.mkdir() method 
  
# importing os module 
import os 
  
# Directory 
# directory = "1"
  
# Parent Directory path 

# os.mkdir(path)   
# Path 
for i in range(0,5):
    path='./pictures'
    path = os.path.join(path, str(i)) 
    os.mkdir(path) 

In [11]:

Kmean.labels_[2]

4

In [12]:
i=0;
count=0;
for key in pics:
    if((pics[key]==vector_key[i]).all()):
        count+=1;
    i+=1;
print(i,count)

160 160


In [13]:
from PIL import Image
i=0;
count=0;
for key in pics:
    path='./pictures'
    path = path+'/'+str(Kmean.labels_[i])
    img = Image.open(os.path.join('./pictures', key))
    img = img.save(path+'/'+key) 
    i+=1;
# print(i,count)for i in range(0,len(Kmean.labels_)):
#     path = os.path.join(path, str(Kmean.labels_[i])) 
#     # os.mkdir(path) 
#     im1 = im1.save(path+) 

In [14]:
i

160